# 딥러닝 레이어의 이해(2) Embedding, Recurrent

## 학습 목표

1. 레이어의 개념을 이해한다.
2. 딥러닝 모델 속 각 레이어(Embedding, RNN, LSTM)의 동작 방식을 이해한다.
3. 데이터의 특성을 고려한 레이어를 설계하고, 이를 Tensorflow로 정의하는 법을 배운다.

### 목차

1. 분포 가설과 분산 표현
2. 단어를 부탁해! Embedding 레이어
3. 순차적인 데이터! Recurrent 레이어 (1) RNN
4. 순차적인 데이터! Recurrent 레이어 (2) LSTM
5. 마무리

## 분포 가설과 분산 표현

* 희소 표현(Sparse Representation) : 벡터의 특정 차원에 단어 혹은 의미를 직접 매핑하는 방식
  - 사과: [ 0, 0 ] , 바나나: [ 1, 1 ] , 배: [ 0, 1 ]

* 단어의 분산 표현(Distributed Representation)
  - 모든 단어들을 고정 차원의 벡터로 
  - 분포 가설(distribution hypothesis) : 유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다
  - 비슷한 맥락의 단어들의 벡터들의 사이거리는 가깝게, 다른 맥락의 단어는 거리가 멀게하는 것
  - 희소 표현과 다르게 단어간의 유사성을 계산할 수 있다.

### Embedding layer

단어 n개를 k차원으로 표현, n x k 형태의 분산 표현

## Embedding Layer

* Weight : 단어의 개수, 단어를 더 깊이 표현(Embedding Size)
* 입력으로 들어온 단어를 분산 표현으로 연결해주는 역할 = Weight에서 특정 행을 읽어오는 것
* Lookup Table

* One-hot Encoding

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
# 위 sentence를 split 한뒤 단어에 해당하는 것을 vocab에서 찾아서 해당 index로 반환
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size) 
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.
print(one_hot.shape)

2022-08-05 12:01:04.142449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 12:01:04.142469: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]
(10, 8)


2022-08-05 12:01:07.976545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 12:01:07.976805: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 12:01:07.976887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-05 12:01:07.976929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-05 12:01:07.976968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

* one_hot encoding 이유 : 단어 단위로 나눠서 벡터 [0,1...]
* one_hot endcoding만 보면 단순 희소 표현이랑 같다.
* Word Embedding은 이런 one_hot encoding으로만 하면 너무 커짐 

In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

In [3]:
print("linear weight")
print(linear.weights)

linear weight
[<tf.Variable 'dense/kernel:0' shape=(8, 2) dtype=float32, numpy=
array([[ 0.19873261,  0.27038038],
       [-0.32390717, -0.06339896],
       [ 0.723773  , -0.4230373 ],
       [-0.2552861 ,  0.33351672],
       [-0.19027472,  0.59898186],
       [-0.01339561, -0.4244931 ],
       [-0.14496893, -0.6882508 ],
       [-0.2884146 , -0.04055977]], dtype=float32)>]


In [4]:
print("one_hot_linear")
print(one_hot_linear)

one_hot_linear
tf.Tensor(
[[ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [-0.32390717 -0.06339896]
 [ 0.723773   -0.4230373 ]
 [-0.2552861   0.33351672]
 [-0.19027472  0.59898186]
 [-0.19027472  0.59898186]
 [-0.19027472  0.59898186]], shape=(10, 2), dtype=float32)


dense Weight에서 input(one_hot Encodding)의 1에 해당하는 부분의 2개의 값을 가져옴  


In [5]:
print("Linear Weight numpy")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight numpy
[[ 0.19873261  0.27038038]
 [-0.32390717 -0.06339896]
 [ 0.723773   -0.4230373 ]
 [-0.2552861   0.33351672]
 [-0.19027472  0.59898186]
 [-0.01339561 -0.4244931 ]
 [-0.14496893 -0.6882508 ]
 [-0.2884146  -0.04055977]]

One-Hot Linear Result
[[ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [ 0.19873261  0.27038038]
 [-0.32390717 -0.06339896]
 [ 0.723773   -0.4230373 ]
 [-0.2552861   0.33351672]
 [-0.19027472  0.59898186]
 [-0.19027472  0.59898186]
 [-0.19027472  0.59898186]]


In [6]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.
print('some_words :', some_words)

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산 표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

some_words : tf.Tensor([[ 3 57 35]], shape=(1, 3), dtype=int32)
Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


Embedding Layer는 결국 단어를 대응만 시킬 뿐이니 미분이 불가능합니다.  
어떤 연산결과를  Embedding Layer에 연결시키는 것은 불가능합니다.  
즉 입력에 바로 Embedding Layer를 연결되게 사용(이때 입력은 one_hot encoding이 이상적)

## RNN

### 이론

인공지능이 예측을 하기 위해서는 요소간의 연관성이 있어야한다.  
고로 딥러닝에서의 시퀀스 데이터는 순차적인 특성을 가진다. 
문장, 영상, 음성은 순차적인 데이터   
이런 순차적인 데이터를 처리하기 위해 고안된 것이 바로 Recurrent Neural Network 또는 Recurrent Layer(RNN) 이다.

* 이전 정보를 반영하는 2가지 방법
  1. (input + prev_hidden) -> hidden -> output
     - Previous hidden : hidden 에서 가져온다.
     - 전부 다 기억할 수 있음
  2. (input _ prev_input ) -> hidden -> output
     - Previous input  : input 에서 가져온다.
     - 바로 직전만을 기억함

* hidden 을 만드는 것이 tanh 라고 합니다
  - sigmoid 보다 tanh 가 기울기의 역전파를 잘 만들어냄
  - sigmoid 미분의 최댓값은 0.25정도, tanh 미분의 최댓값은 1이므로 tanh 가 gradient vanishing에 더 강하다.
* 자동 완성사례에서는 input이 그 전의 output을 사용합니다.(2번째 뉴런의 input이 1번째의 output(1번째 글자))  
    (단 여기서 예시를 1번째 글자를 넣으면 나머지 글자를 나오게 하는 모델입니다.)

* RNN의 입력으로 들어가는 모든 단어만큼 Weight를 만드는 것이 아님.
* (입력차원, 출력차원의 크기의) 하나의 Weight를 순차적으로 업데이트

<img src = 'GradientLoss.png' width = 50%, height = 50%>

기울기 소실(Vanishing Gradient) : What의 정보(앞부분의 정보)가 가면 갈수록 희석되는 문제점

### 코드

Embedding Layer 와 RNN Layer의 구체적인 형태를 보여주는 코드  
[TensorFlow RNN](https://www.tensorflow.org/guide/keras/rnn?hl=ko)

In [7]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


* 문장의 긍정/부정을 구분하는 것은 문장을 모두 읽은 뒤, 최종 Step의 Output만 확인해도 판단 가능
* 문장을 생성하는 경우는 모든 step에 대한 Output이 필요
* 위는 tf.keras.layers.SimpleRNN 레이어의 return_sequences 인자 조절

In [8]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


LSTM 의 Weight 의 크기가 RNN 의 4배 입니다.

## LSTM

* RNN 은 장기 의존성(Long-Term Dependency) 을 잘 다루지 못한다.  
* LSTM 은 Long Short-Term Memory의 약어로 기울기 소실 문제를 해결하기 위해 고안된 RNN Layer  
* Deep Learning Network 는 각 가중치의 미분을 구해 업데이트하는 backpropagation을 통해 학습함.
* RNN은 입력되는 문장의 길이가 길수록 초기에 입력된 단어들의 미분 값이 매우 작아지거나 커지는 현상
* 너무 작아지는 것 : Vanishing Gradient : 학습이 제대로 안됨 - RNN 구조 변경(tanh 사용 등)을 통해 방지
* 너무 커지는 것 : Exploding Gradient   : 학습이 불안정함 - Gradient clipping을 통해 방지

* cell state = c
* hidden state = h

<img src = 'LSTM1.png' width = 50%, height = 50%>

* Weight에서 나오는 Gate
1. Input gate : 셀에 적을 것인지? - i[0,1] - sigmoid
2. Forget gate : 셀을 지울 것인지? - f[0,1] - sigmoid
3. Ouput gate : 셀을 얼마나 드러낼 것인지? - o[0,1] - sigmoid
4. Gate gate(?): 셀에 얼마나 적을 것인지 - g[-1,1] - tanh

작동 방식

<img src = 'LSTM2.png' width = 50%, height = 50%>

기존의 RNN의 Weight 하나를 계속 바꾸는 시스템(exploding, vanishing gradient 유발)과 다르게  
LSTM은 f가 task마다 바뀌니까 더 좋음.  

### GRU

* Gated Recurrent Unit
* cell state 와 hidden state 를 합침.
* forget gate 와 input gate 를 통합.
* 학습하는 가중치의 양이 줄어든다.

### 양방향(Bidirectional) RNN

* 한쪽 방향으로만 학습할 경우 target의 예측하기 힘든 경우
* tf.keras.layers.Bidirectional() 로 Layer를 감싸주면 됨

코드

In [9]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


기본 RNN의 크기가 2배가 되었습니다.